<a href="https://colab.research.google.com/github/eth212/QMIND/blob/master/qmind_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()
#upload your file here
#gives an error the first time so run this twice
#make sure your file is called "colab.xlsx"

In [0]:
!pip install tensorflow-gpu==2.0.0rc numpy tqdm matplotlib --no-cache-dir

     |████████████████████████████████| 380.5MB 26.8MB/s 
     |████████████████████████████████| 4.3MB 33.1MB/s 
     |████████████████████████████████| 501kB 49.2MB/s 


In [0]:
import tensorflow as tf
from tensorflow import keras
tf.__version__

'2.0.0-rc0'

In [0]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib
from matplotlib import pyplot as plt
#! pip install tensorflow-gpu==2.0.0 

In [0]:
buffer = pd.read_excel("colab.xlsx")
df = pd.DataFrame(buffer)

In [0]:
#alternate df
df_alt = pd.DataFrame()
df_alt = df
#only ever run this once (at beginning)

In [0]:
def get_counts(series):
  X = series.tolist()
  count = Counter(np.array(X))
  tuple_list = count.most_common()
  return tuple_list

In [0]:
def range2(np_array):
  max = np.max(np_array)
  min = np.min(np_array)
  return max - min

In [0]:
def one_hot_encode(series_original, df_destination):
  new_df = pd.get_dummies(series_original)
  df_destination = pd.concat([df_destination,new_df], axis=1)
  return df_destination

In [0]:
def get_data(df):
  data = []
  list_cols = list(df.columns)
  for j in range(len(df)):
    temp = []   
    for i in range(len(df.columns)):
      val = df[list_cols[i]].iloc[j]  
      temp.append(val)
    data.append(temp)
  return data

In [0]:
def seperate(y_index, data_set):
  X = []
  Y = df[df.columns[y_index]].tolist()
  for i in range(len(data_set)):
    cols = len(data_set[i])
    X.append(data_set[i][(y_index + 1):cols])
  return Y ,X

In [0]:
df_alt

,Classification,Datasource,SerialNumber,Year,Make,Model,Salesprice,SaleDate,Location,AuctionCompany,Mileage,Engine,HP,Suspension,Sleeper,Trans,Spd,Axles,TransactionType,mNotes
0,Tandem Axle,DataSource2,1YDB48052661,2000.0,FREIGHTLINER,CLASSIC XL,16052.0,2009-12-09,"PELZER, SC",198 Auction Company Inc.,NaN,DET S60,NaN,NaN,CONDO SLEEPER,NaN,9.0,NaN,NaN,"DET S60 ENG, 9 SPD TRANS, JAKE BRAKE, 12,000 L..."
1,Tandem Axle,DataSource2,2YDF89395692,2000.0,FREIGHTLINER,CLASSIC XL,13532.0,2009-12-09,"PELZER, SC",198 Auction Company Inc.,NaN,DET S60,NaN,NaN,CONDO SLEEPER,NaN,9.0,NaN,NaN,"DET S60 ENG, 9 SPD TRANS, JAKE BRAKE, 12,000 L..."
2,Tandem Axle,DataSource2,36DW0139920,2006.0,FREIGHTLINER,CLASSIC FLD12064T,44726.0,2009-12-03,"MONTREAL, QC",1576 Auction Company Inc.,NaN,CAT C15 550 HP,550.0,AIR RIDE,"40"""" FLAT TOP INTEGRAL SLEEPER",NaN,18.0,NaN,NaN,"CAT C15 550 HP ENG, ENG BRAKE, EATON FULLER 18..."
3,Tandem Axle,DataSource2,4YPB9338177,2000.0,FREIGHTLINER,CLASSIC XL,14612.0,2009-10-29,"LITTLE ROCK, AR",6356 Auction Company Inc.,NaN,CUMMINS N14 DSL,NaN,AIR RIDE,"60"""" SLEEPER",NaN,13.0,NaN,NaN,"CUMMINS N14 DSL ENG, 13 SPD TRANS, JAKE BRAKE,..."
4,Tandem Axle,DataSource2,01LB87143365,2001.0,FREIGHTLINER,CLASSIC XL,20282.0,2009-10-29,"LITTLE ROCK, AR",6356 Auction Company Inc.,NaN,12.7L 500 HP DSL,500.0,AIR RIDE,"72"""" SLEEPER",NaN,10.0,NaN,NaN,"12.7L 500 HP DSL ENG, 10 SPD TRANS, JAKE BRAKE..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154838,Conventional,DataSource1,5PW009133458,1993.0,MACK,CH612,10382.0,2019-06-25,NEWBURGH NY,NaN,884000,Mack E7,300.0,Spring,Day Cab,7 Speed Transmission,7.0,Single,NaN,"1993 MACK CH612 Non-Sleeper Conventional, 884,..."
154839,Conventional,DataSource1,XMW009842591,1991.0,MACK,CH612,10382.0,2019-06-25,NEWBURGH NY,NaN,0,Mack E7,300.0,Spring,Day Cab,RTX-14609B Transmission,NaN,Single,NaN,"1991 MACK CH612 Non-Sleeper Conventional, Mack..."
154840,Conventional,DataSource1,XMW006802607,1991.0,MACK,CH612,10382.0,2019-06-25,NEWBURGH NY,NaN,0,Mack E7,300.0,Spring,Day Cab,RTX-14609B Transmission,NaN,Single,NaN,"1991 MACK CH612 Non-Sleeper Conventional, Mack..."
154841,Conventional,DataSource1,7MW003702595,1991.0,MACK,CH612,10382.0,2019-06-25,NEWBURGH NY,NaN,0,Mack E7,300.0,Spring,Day Cab,RTX-14609B Transmission,NaN,Single,NaN,"1991 MACK CH612 Non-Sleeper Conventional, Mack..."


In [0]:
#getting all the data
#dropping the columns that aren't in use for the model

#run this when you want to reset the dataframe back to original (faster than re-reading file)
df = df_alt

#df = df.drop( columns = [ 'Classification', 'Datasource', 'SerialNumber', 'Make', 'AuctionCompany', 'Mileage','HP', 'Suspension', 'SaleDate','Sleeper', 'Trans', 'Spd', 'Axles','TransactionType', 'mNotes','Unnamed: 0'])
df = df.drop( columns = [ 'Classification', 'Datasource', 'SerialNumber', 'Make', 'AuctionCompany', 'Mileage','HP', 'Suspension', 'SaleDate','Sleeper', 'Trans', 'Spd', 'Axles','TransactionType', 'mNotes'])
df = df.dropna()



In [0]:
df.columns

Index(['Year', 'Model', 'Salesprice', 'Location', 'Engine'], dtype='object')

In [0]:
#refining data based number of times it appears in dataset

model_list = get_counts(df["Model"])
num_models = 20
list_of_models_to_use = model_list[0:num_models] #the number after 0 in  [0:20] dictates the number of models to use (popular first)
models = []
for i in range(num_models):
  models.append(list_of_models_to_use[i][0])
df = df[df["Model"].isin(models)]
df

,Year,Model,Salesprice,Location,Engine
15,2010.0,386 ULTRACAB,70232.0,SOUTH SIOUX CITY NE,Cummins ISX
120,1997.0,FLD120,17582.0,"PLEASANTON, CA",CUMMINS N14
121,1998.0,FLD120,19832.0,"PLEASANTON, CA",CUMMINS N14
122,2000.0,FLD120,18032.0,"PLEASANTON, CA",DET SERIES 60
123,2000.0,FLD120,18932.0,"PLEASANTON, CA",DET SERIES 60 500 HP
...,...,...,...,...,...
154815,2003.0,CH613,15332.0,NEWBURGH NY,Mack E7 E-Tech
154825,1995.0,CH613,17132.0,NEWBURGH NY,Mack E7
154832,1994.0,CH613,17582.0,NEWBURGH NY,Mack E7
154833,1994.0,CH613,17582.0,NEWBURGH NY,Mack E7


In [0]:
#refining data with Year of make
year_list = get_counts(df["Year"])
list_of_years_to_use = year_list[0:1][0]  #using the 1 most popular year (2004)
df = df[df["Year"].isin(list_of_years_to_use)]
df

,Year,Model,Salesprice,Location,Engine
161,2005.0,FLD120,45032.0,"SACRAMENTO, CA",MERCEDES BENZ OM460 450 HP
203,2005.0,CL120 COLUMBIA,25232.0,CHARLOTTE NC,Mercedes 12.8 Liter
204,2005.0,9400I,28382.0,CHARLOTTE NC,Cummins ISX
205,2005.0,9400,23882.0,CHARLOTTE NC,Cummins ISX
206,2005.0,9400I,25232.0,"DENVER, CO",CUMMINS ISX435ST 15.0L
...,...,...,...,...,...
154303,2005.0,C120 CENTURY CLASS,13532.0,SOUTH SIOUX CITY NE,Detroit 12.7 Liter
154304,2005.0,CL120 COLUMBIA,10382.0,SOUTH SIOUX CITY NE,Caterpillar C15
154305,2005.0,CL120 COLUMBIA,10382.0,SOUTH SIOUX CITY NE,Caterpillar C15
154756,2005.0,9400I,19832.0,TUNICA MS,Caterpillar C13


In [0]:
#refining the engines used
engine_list = get_counts(df["Engine"])
num_engines = 1
list_of_engines = engine_list[0:num_engines] 
eng = []
for i in range(num_engines):
  eng.append(list_of_engines[i][0])
df = df[df["Engine"].isin(eng)]
df

,Year,Model,Salesprice,Location,Engine
204,2005.0,9400I,28382.0,CHARLOTTE NC,Cummins ISX
205,2005.0,9400,23882.0,CHARLOTTE NC,Cummins ISX
208,2005.0,9400I,23432.0,CHARLOTTE NC,Cummins ISX
1429,2005.0,9400I,25682.0,PEORIA IL,Cummins ISX
1433,2005.0,9400I,25232.0,PEORIA IL,Cummins ISX
...,...,...,...,...,...
150233,2005.0,9400I,20732.0,PEORIA IL,Cummins ISX
150234,2005.0,9200I,15782.0,PEORIA IL,Cummins ISX
150608,2005.0,9400I,14432.0,TUNICA MS,Cummins ISX
150609,2005.0,9400I,13532.0,TUNICA MS,Cummins ISX


In [0]:
#using 5 locations 
location_list = get_counts(df["Location"])
num_locations = 5
list_of_locations = location_list[0:num_locations]
loc = []
for i in range(num_locations):
  loc.append(list_of_locations[i][0])
df = df[df["Location"].isin(loc)]
df

,Year,Model,Salesprice,Location,Engine
1429,2005.0,9400I,25682.0,PEORIA IL,Cummins ISX
1433,2005.0,9400I,25232.0,PEORIA IL,Cummins ISX
1435,2005.0,9400I,24782.0,PEORIA IL,Cummins ISX
1437,2005.0,9400I EAGLE,22982.0,PEORIA IL,Cummins ISX
3326,2005.0,9400I,57632.0,SOUTH SIOUX CITY NE,Cummins ISX
...,...,...,...,...,...
150233,2005.0,9400I,20732.0,PEORIA IL,Cummins ISX
150234,2005.0,9200I,15782.0,PEORIA IL,Cummins ISX
150608,2005.0,9400I,14432.0,TUNICA MS,Cummins ISX
150609,2005.0,9400I,13532.0,TUNICA MS,Cummins ISX


In [0]:
#one hot encoding locations into dummy variables
df = one_hot_encode(df["Model"], df)
df = one_hot_encode(df["Location"], df)
df = one_hot_encode(df["Engine"], df)

In [0]:
df

,Year,Model,Salesprice,Location,Engine,378,379,379 EXTENDED HOOD,379 ULTRACAB EXTENDED HOOD,387,9200I,9400,9400I,9400I EAGLE,T-600 AEROCAB,W-900L AEROCAB,FORT WORTH TX,LAS VEGAS NV,PEORIA IL,SOUTH SIOUX CITY NE,TUNICA MS,Cummins ISX
1429,2005.0,9400I,25682.0,PEORIA IL,Cummins ISX,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1
1433,2005.0,9400I,25232.0,PEORIA IL,Cummins ISX,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1
1435,2005.0,9400I,24782.0,PEORIA IL,Cummins ISX,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1
1437,2005.0,9400I EAGLE,22982.0,PEORIA IL,Cummins ISX,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1
3326,2005.0,9400I,57632.0,SOUTH SIOUX CITY NE,Cummins ISX,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150233,2005.0,9400I,20732.0,PEORIA IL,Cummins ISX,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1
150234,2005.0,9200I,15782.0,PEORIA IL,Cummins ISX,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
150608,2005.0,9400I,14432.0,TUNICA MS,Cummins ISX,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1
150609,2005.0,9400I,13532.0,TUNICA MS,Cummins ISX,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1


In [0]:
df = df.drop(columns = ["Location", "Model", "Year", "Engine", "Cummins ISX"])

In [0]:
#df = df.drop(columns = ["Salesprice"])

In [0]:
#getting data into lists
data_set = get_data(df)


In [0]:
data_set[0]

[25682.0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]

In [0]:
Y, X = seperate(0,data_set)

In [0]:
X[0,:]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], dtype=uint8)

In [0]:
#resizes the arrays
# global num_cols
# num_cols = len(df.columns) -1
# global num_rows
# num_rows = len(X)
# X = np.resize(X, (num_rows,num_cols))

In [0]:
df

,Salesprice,378,379,379 EXTENDED HOOD,379 ULTRACAB EXTENDED HOOD,387,9200I,9400,9400I,9400I EAGLE,T-600 AEROCAB,W-900L AEROCAB,FORT WORTH TX,LAS VEGAS NV,PEORIA IL,SOUTH SIOUX CITY NE,TUNICA MS
1429,25682.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1433,25232.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1435,24782.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1437,22982.0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
3326,57632.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150233,20732.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
150234,15782.0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
150608,14432.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
150609,13532.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [0]:
X = np.array(X)
if (len(X) != len(Y)):
  print("error")

In [0]:
#normalizing Y

#divide by largest value
#see if you can get the model to predict accurate values without normalizing Y
# Y = np.array(Y)
# Y = Y/np.max(Y)

In [0]:
Y = np.array(Y).reshape(len(Y),1)
Y

array([[25682.],
       [25232.],
       [24782.],
       ...,
       [14432.],
       [13532.],
       [56282.]])

In [0]:
#splitting the data up 
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [0]:
# Feature Scaling
'''from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
'''


'from sklearn.preprocessing import StandardScaler\nsc = StandardScaler()\nX_train = sc.fit_transform(X_train)\nX_test = sc.transform(X_test)\n'

In [0]:
#make sure all values are scaled 
X_train[0:10]

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]], dtype=uint8)

In [0]:
# Importing the Keras libraries and packages
# import keras
# from keras.models import Sequential
# from keras.layers import Dense

Using TensorFlow backend.


In [0]:
# #Initializing Neural Network
# model = Sequential()


In [0]:
# # Adding the input layer and the first hidden layer
# #the model has 8 features

# #units are the number of nodes (neurons) per layer and should be changed

# model.add(Dense(activation = 'relu', input_dim = num_cols ,units=80, use_bias=True,kernel_initializer="uniform" ))
# # Adding the second hidden layer
# model.add(Dense(activation = 'relu', units=200, kernel_initializer="uniform" )) #try tanh
# # Adding the output layer
# model.add(Dense(activation = "sigmoid", units=1 ))
# #look into changing the activation of this layer to sigmoid and then scaling the values to match with the range of Y

In [0]:
num_cols = np.shape(X)[1]
model = keras.Sequential([
    keras.layers.Dense(20, activation='relu', input_dim=num_cols),
    keras.layers.Dense(40, activation='relu'),
    keras.layers.Dense(80, activation='relu'),
    keras.layers.Dense(1, activation='linear')
])

In [0]:
model.compile(optimizer = 'adam', loss = ["mean_squared_error"], metrics = ["mse"])

In [0]:
model.fit(X_train, Y_train, batch_size = 1, epochs = 15)

Train on 1158 samples
Epoch 1/15
1158/1158 [==============================] - 4s 4ms/sample - loss: 772034761.3130 - mse: 772034432.0000
Epoch 2/15
1158/1158 [==============================] - 4s 3ms/sample - loss: 271591440.5123 - mse: 271591520.0000
Epoch 3/15
1158/1158 [==============================] - 4s 3ms/sample - loss: 239889421.7437 - mse: 239889376.0000
Epoch 4/15
1158/1158 [==============================] - 4s 3ms/sample - loss: 219538493.1204 - mse: 219538384.0000
Epoch 5/15
1158/1158 [==============================] - 4s 3ms/sample - loss: 206209206.0021 - mse: 206209152.0000
Epoch 6/15
1158/1158 [==============================] - 4s 3ms/sample - loss: 198885888.8390 - mse: 198886048.0000
Epoch 7/15
1158/1158 [==============================] - 4s 3ms/sample - loss: 192853624.0479 - mse: 192853632.0000
Epoch 8/15
1158/1158 [==============================] - 4s 3ms/sample - loss: 190009263.8465 - mse: 190009232.0000
Epoch 9/15
1158/1158 [==============================] - 4s

In [0]:
Y_pred=model.predict(X_test)
Y_pred

array([[32567.068],
       [34189.582],
       [32567.068],
       [32775.32 ],
       [33819.344],
       [36654.406],
       [33101.1  ],
       [33101.1  ],
       [38025.836],
       [33819.344],
       [36654.406],
       [33101.1  ],
       [33475.242],
       [35164.344],
       [32775.32 ],
       [34603.62 ],
       [61806.496],
       [33475.242],
       [32775.32 ],
       [32567.068],
       [54955.15 ],
       [32775.32 ],
       [32775.32 ],
       [35912.945],
       [36216.855],
       [76869.39 ],
       [33819.344],
       [32775.32 ],
       [33475.242],
       [60308.258],
       [33101.1  ],
       [36654.406],
       [33101.1  ],
       [32567.068],
       [34189.582],
       [34189.582],
       [60242.453],
       [32567.068],
       [32567.068],
       [38558.316],
       [32567.068],
       [33101.1  ],
       [32775.32 ],
       [34603.62 ],
       [35164.344],
       [33101.1  ],
       [33475.242],
       [33819.344],
       [36654.406],
       [32567.068],


In [0]:
Y_test

array([[ 23882.],
       [ 30182.],
       [ 32882.],
       [ 45932.],
       [ 55382.],
       [ 29282.],
       [ 27932.],
       [ 28382.],
       [ 36932.],
       [ 10382.],
       [ 14432.],
       [ 27212.],
       [ 23432.],
       [ 32882.],
       [ 36482.],
       [ 33332.],
       [ 41432.],
       [ 27932.],
       [ 41432.],
       [ 32432.],
       [ 30632.],
       [ 43232.],
       [ 31082.],
       [ 11282.],
       [ 37832.],
       [ 86432.],
       [ 27032.],
       [ 16682.],
       [ 36032.],
       [ 64382.],
       [ 27212.],
       [ 44132.],
       [ 33782.],
       [ 25682.],
       [ 33782.],
       [ 31082.],
       [ 55382.],
       [ 33332.],
       [ 27032.],
       [ 44582.],
       [ 43232.],
       [ 34682.],
       [ 30182.],
       [ 45032.],
       [ 33782.],
       [ 27482.],
       [ 49982.],
       [ 31082.],
       [ 51332.],
       [ 28382.],
       [ 72032.],
       [ 41432.],
       [ 29282.],
       [ 30632.],
       [ 34232.],
       [ 2

In [0]:
# # Compiling Neural Network
# model.compile(optimizer = 'adam', loss = ["binary_crossentropy"], metrics = ["mse", "mae"])
# #not sure adam is the best optimizer to use



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# Fitting our model 
#batch size is only 1 right now because model is not fully functional
# model.fit(X_train, Y_train, batch_size = 1, epochs = 5)

Train on 1158 samples
Epoch 1/5
1158/1158 [==============================] - 5s 4ms/sample - loss: 1442216096.6762 - mse: 1442215680.0000
Epoch 2/5
1158/1158 [==============================] - 3s 3ms/sample - loss: 540215726.9585 - mse: 540215872.0000
Epoch 3/5
1158/1158 [==============================] - 4s 3ms/sample - loss: 288374348.1004 - mse: 288374432.0000
Epoch 4/5
1158/1158 [==============================] - 4s 3ms/sample - loss: 269800156.2850 - mse: 269800224.0000
Epoch 5/5
1158/1158 [==============================] - 3s 3ms/sample - loss: 256410211.6264 - mse: 256410032.0000


In [0]:
# model.predict(X_train[0:5])

In [0]:
# Y[0:5]

array([0.00025682, 0.00025232, 0.00024782, 0.00022982, 0.00057632])